In [1]:
import os
import shutil
import pandas as pd
from openpyxl import load_workbook

def convert_percentage_values(df):
    def convert_value(value):
        if isinstance(value, (int, float)):
            return value * 100 if 0 <= value < 1 else value
        return value
    
    df["VALOR"] = df["VALOR"].apply(convert_value)
    return df

def add_summary_sheet(file_path, no_value_responsibles, high_completion_responsibles):
    wb = load_workbook(file_path)
    
    # Adicionar nova aba e renomeá-la
    summary_sheet = wb.create_sheet("Resumo")
    
    # Adicionar cabeçalhos
    summary_sheet.cell(row=1, column=1, value="Responsáveis sem Valor")
    summary_sheet.cell(row=1, column=2, value="Responsáveis com 80%+ Preenchimento")
    
    # Preencher a lista de responsáveis sem valor
    for index, responsible in enumerate(no_value_responsibles):
        summary_sheet.cell(row=index+2, column=1, value=responsible)
    
    # Preencher a lista de responsáveis com 80%+ preenchimento
    for index, responsible in enumerate(high_completion_responsibles):
        summary_sheet.cell(row=index+2, column=2, value=responsible)
    
    # Salvar e fechar a planilha
    wb.save(file_path)
    wb.close()

# Caminho e nome da planilha original
planilha_original = "C:/Users/fcsantos/OneDrive - BERNHOEFT CONTADORES ASSOCIADOS LTDA/_Inteligência_Empresarial/Plan_Estratégico/BI/BI_FELIPE/BI 2023/RAR.xlsx"

# Caminho e nome da planilha copiada
planilha_copia = "C:/Users/fcsantos/OneDrive - BERNHOEFT CONTADORES ASSOCIADOS LTDA/_Inteligência_Empresarial/Plan_Estratégico/BI/BI_FELIPE/BI 2023/RAR_copia.xlsx"

# Caminho e nome da planilha concatenada
planilha_concatenada = "C:/Users/fcsantos/OneDrive - BERNHOEFT CONTADORES ASSOCIADOS LTDA/_Inteligência_Empresarial/Plan_Estratégico/BI/BI_FELIPE/BI 2023/RAR_Concatenada.xlsx"

# Copiar a planilha original para criar uma cópia temporária
shutil.copy(planilha_original, planilha_copia)

# Carregar a planilha copiada
wb = load_workbook(planilha_copia)

# Ler o valor do período na célula A1 da aba "PERIODO"
periodo_sheet = wb["PERIODO"]
periodo_desejado = periodo_sheet["A1"].value

# Ignorar as abas "MENU", "Plan1" e "PERIODO"
sheets_to_read = [sheet for sheet in wb.sheetnames if sheet not in ["MENU", "Plan1", "PERIODO", "Validação de Dados_COLAB", "Vinicius Ferreira de Souza", "INICIATIVAS_COLAB"]]

df_concatenado = pd.DataFrame()
for sheet in sheets_to_read:
    try:
        df = pd.read_excel(planilha_copia, sheet_name=sheet)
        df_concatenado = pd.concat([df_concatenado, convert_percentage_values(df)])
    except Exception as e:
        print(f"Erro ao ler a aba {sheet}. Erro: {str(e)}")
        continue

# Substituir valores vazios por 'x' na coluna "VALOR"
df_concatenado["VALOR"].fillna("x", inplace=True)

# Filtrar os dados pelo período desejado
df_filtrado = df_concatenado[df_concatenado["PERÍODO"] == periodo_desejado]

# Agrupar os dados filtrados por responsável e calcular a porcentagem de linhas preenchidas na coluna "VALOR"
grouped_data = df_filtrado.groupby("RESP")["VALOR"].agg(["count", "size"])
grouped_data["completion_percentage"] = (grouped_data["count"] / grouped_data["size"]) * 100

# Filtrar responsáveis que não possuem nenhum valor relacionado na coluna "VALOR"
no_value_responsibles = grouped_data[grouped_data["count"] == 0].index.tolist()

# Filtrar responsáveis com pelo menos 80% das linhas preenchidas na coluna "VALOR"
high_completion_responsibles = grouped_data[grouped_data["completion_percentage"] >= 80].index.tolist()

# Salvar o DataFrame concatenado e filtrado como uma nova planilha
df_filtrado.to_excel(planilha_concatenada, index=False)

# Adicionar a tabela de resumo na planilha "RAR_Concatenada.xlsx"
add_summary_sheet(planilha_concatenada, no_value_responsibles, high_completion_responsibles)

# Fechar e remover a planilha copiada
wb.close()
os.remove(planilha_copia)

print("Planilhas concatenadas e resumo adicionado com sucesso!")


C:\Users\fcsantos\Anaconda3\lib\site-packages\openpyxl\reader\excel.py:228: UserWarning: Data Validation extension is not supported and will be removed
  ws_parser.bind_all()


Planilhas concatenadas e resumo adicionado com sucesso!
